# Analysis of Natural Amenities and Housing Costs

I want to relate housing prices to the natural amenities found in a given county. I will be using a dataset called the Natural Amenities Scale, which can be found [here](https://www.ers.usda.gov/data-products/natural-amenities-scale.aspx)

I will also be using a dataset of 

In [14]:
import pandas as pd
import numpy as np

In [250]:
# importing natural amenity csv
nat = pd.read_csv('/Users/patricknorman/Documents/GitHub/metis-project-2/data/nat_amen_clean.csv')

In [251]:
# verifying sucessful import
nat.head()

,FIPS,COMBINED_COUNTY_INDEX,STATE,COUNTY,CENSUS_DIV,RUC,UIC,JAN_TEMP,JAN_SUN,JULY_TEMP,...,WATER,LN_WATER,JAN TEMP - Z,JAN SUN - Z,JUL TEMP - Z,JUL HUM - Z,TOPOG - Z,LN WATER AREA - Z,NAT_AMENITY,RANK
0,1001,1001,AL,AUTAUGA,6,2,2,47.4,130,81.0,...,1.40,4.944,1.20277,-0.64758,-0.13206,-0.68502,0.77701,0.26585,0.78,4
1,1003,1003,AL,BALDWIN,6,2,2,51.9,152,80.6,...,21.24,7.661,1.57504,0.01482,0.36308,-1.09576,-0.73966,1.70428,1.82,4
2,1005,1005,AL,BARBOUR,6,6,5,48.8,152,81.0,...,2.17,5.378,1.31859,0.01482,-0.01148,-0.89039,-0.73966,0.49570,0.19,4
3,1007,1007,AL,BIBB,6,6,6,44.8,130,80.8,...,0.50,3.916,0.98768,-0.64758,-0.30223,-0.68502,0.77701,-0.27812,-0.15,3
4,1009,1009,AL,BLOUNT,6,2,2,41.9,130,78.4,...,0.76,4.336,0.74778,-0.64758,0.09330,-0.68502,0.77701,-0.05592,0.23,4


In [252]:
#clarifying some column names
nat.rename(columns={'RUC':'RUC_93','UIC':'UIC_93'},inplace=True)

In [5]:
# examining shape
nat.shape

(3111, 22)

I've found a good housing cost dataset for 2013 [here](https://data.census.gov/cedsci/table?q=home%20values%202013%20county&t=Housing&g=0100000US.050000&tid=ACSST1Y2013.S2506&hidePreview=true). I just have to import it and wrangle with the totally different format...

In [258]:
# importing housing df
housing_raw = pd.read_csv('/Users/patricknorman/Documents/GitHub/metis-project-2/data/2013_housing.csv')
housing_raw

,Label,"Fairbanks North Star Borough, Alaska!!Owner-occupied housing units with a mortgage!!Estimate","Fairbanks North Star Borough, Alaska!!Owner-occupied housing units with a mortgage!!Margin of Error","Apache County, Arizona!!Owner-occupied housing units with a mortgage!!Estimate","Apache County, Arizona!!Owner-occupied housing units with a mortgage!!Margin of Error","Yuba County, California!!Owner-occupied housing units with a mortgage!!Estimate","Yuba County, California!!Owner-occupied housing units with a mortgage!!Margin of Error","Bulloch County, Georgia!!Owner-occupied housing units with a mortgage!!Estimate","Bulloch County, Georgia!!Owner-occupied housing units with a mortgage!!Margin of Error","Catoosa County, Georgia!!Owner-occupied housing units with a mortgage!!Estimate",...,"Harford County, Maryland!!Owner-occupied housing units with a mortgage!!Margin of Error","Darlington County, South Carolina!!Owner-occupied housing units with a mortgage!!Estimate","Darlington County, South Carolina!!Owner-occupied housing units with a mortgage!!Margin of Error","Kennebec County, Maine!!Owner-occupied housing units with a mortgage!!Estimate","Kennebec County, Maine!!Owner-occupied housing units with a mortgage!!Margin of Error","Rutherford County, North Carolina!!Owner-occupied housing units with a mortgage!!Estimate","Rutherford County, North Carolina!!Owner-occupied housing units with a mortgage!!Margin of Error","Pasco County, Florida!!Owner-occupied housing units with a mortgage!!Estimate","Pasco County, Florida!!Owner-occupied housing units with a mortgage!!Margin of Error","Jasper County, Missouri!!Owner-occupied housing units with a mortgage!!Estimate"
0,Owner-occupied housing units with a mortgage,"13,579","±1,488","4,119",±842,"9,869","±1,344","7,253","±1,067","10,599",...,"±2,512","8,837","±1,243","20,626","±1,585","8,552","±1,169","72,592","±2,756","17,315"
1,VALUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Less than $50,000",1.3%,±1.2,16.5%,±6.5,4.9%,±3.3,8.7%,±5.0,3.2%,...,±1.0,15.9%,±6.3,4.2%,±1.9,14.5%,±6.5,9.5%,±1.4,9.7%
3,"$50,000 to $99,999",0.8%,±1.2,21.4%,±7.5,13.3%,±5.4,11.1%,±5.4,15.2%,...,±1.2,28.9%,±7.3,14.2%,±3.9,29.1%,±8.6,22.9%,±2.0,31.7%
4,"$100,000 to $149,999",10.1%,±4.7,12.9%,±7.7,18.5%,±6.4,39.9%,±10.8,35.9%,...,±1.5,23.3%,±7.8,28.5%,±5.0,18.1%,±7.7,19.8%,±2.3,21.7%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Less than $800,3.3%,±2.3,37.3%,±13.6,11.7%,±3.8,27.9%,±8.1,21.2%,...,±0.8,73.5%,±5.9,6.0%,±2.3,55.5%,±7.7,23.7%,±2.1,39.9%
71,"$800 to $1,499",8.9%,±4.1,16.9%,±10.1,26.2%,±7.3,36.2%,±9.9,43.6%,...,±2.1,13.7%,±6.0,22.8%,±4.1,22.3%,±6.7,25.0%,±2.2,33.5%
72,"$1,500 or more",78.4%,±6.3,8.5%,±7.1,61.2%,±7.7,33.7%,±8.9,33.0%,...,±2.2,9.2%,±4.0,70.7%,±4.4,19.7%,±5.9,48.8%,±2.4,24.3%
73,No real estate taxes paid,9.4%,±4.0,37.2%,±10.1,0.8%,±1.4,2.2%,±1.8,2.1%,...,±1.3,3.7%,±3.4,0.5%,±0.6,2.6%,±2.3,2.4%,±0.7,2.2%


In [259]:
housing_raw.columns

Index(['Label',
       'Fairbanks North Star Borough, Alaska!!Owner-occupied housing units with a mortgage!!Estimate',
       'Fairbanks North Star Borough, Alaska!!Owner-occupied housing units with a mortgage!!Margin of Error',
       'Apache County, Arizona!!Owner-occupied housing units with a mortgage!!Estimate',
       'Apache County, Arizona!!Owner-occupied housing units with a mortgage!!Margin of Error',
       'Yuba County, California!!Owner-occupied housing units with a mortgage!!Estimate',
       'Yuba County, California!!Owner-occupied housing units with a mortgage!!Margin of Error',
       'Bulloch County, Georgia!!Owner-occupied housing units with a mortgage!!Estimate',
       'Bulloch County, Georgia!!Owner-occupied housing units with a mortgage!!Margin of Error',
       'Catoosa County, Georgia!!Owner-occupied housing units with a mortgage!!Estimate',
       ...
       'Harford County, Maryland!!Owner-occupied housing units with a mortgage!!Margin of Error',
       'Darli

In [264]:
# formatting housing df
housing = housing_raw.transpose()
housing.columns = housing.iloc[0]
housing.drop(labels=['Label'],inplace=True)
housing.reset_index(inplace=True)
housing.rename(columns={
    'index':'COUNTY',
    'Owner-occupied housing units with a mortgage':'N'},
               inplace=True)
housing

Label,COUNTY,N,VALUE,"Less than $50,000","$50,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 to $299,999","$300,000 to $499,999","$500,000 or more",...,Less than 20 percent,20 to 29 percent,30 percent or more,Zero or negative income,REAL ESTATE TAXES,Less than $800,"$800 to $1,499","$1,500 or more",No real estate taxes paid,Median (dollars)
0,"Fairbanks North Star Borough, Alaska!!Owner-oc...","13,579",NaN,1.3%,0.8%,10.1%,28.9%,41.1%,12.1%,5.6%,...,N,N,N,N,NaN,3.3%,8.9%,78.4%,9.4%,"2,969"
1,"Fairbanks North Star Borough, Alaska!!Owner-oc...","±1,488",NaN,±1.2,±1.2,±4.7,±6.7,±6.4,±5.2,±4.0,...,N,N,N,N,NaN,±2.3,±4.1,±6.3,±4.0,±272
2,"Apache County, Arizona!!Owner-occupied housing...","4,119",NaN,16.5%,21.4%,12.9%,27.0%,11.1%,5.1%,5.9%,...,26.4%,4.5%,0.0%,0.8%,NaN,37.3%,16.9%,8.5%,37.2%,690
3,"Apache County, Arizona!!Owner-occupied housing...",±842,NaN,±6.5,±7.5,±7.7,±12.5,±6.9,±4.6,±5.1,...,±9.3,±5.4,±4.0,±0.9,NaN,±13.6,±10.1,±7.1,±10.1,±255
4,"Yuba County, California!!Owner-occupied housin...","9,869",NaN,4.9%,13.3%,18.5%,26.3%,23.2%,10.9%,3.0%,...,N,N,N,N,NaN,11.7%,26.2%,61.2%,0.8%,"1,856"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,"Rutherford County, North Carolina!!Owner-occup...","8,552",NaN,14.5%,29.1%,18.1%,15.5%,11.4%,6.7%,4.7%,...,21.7%,7.3%,1.7%,0.9%,NaN,55.5%,22.3%,19.7%,2.6%,725
995,"Rutherford County, North Carolina!!Owner-occup...","±1,169",NaN,±6.5,±8.6,±7.7,±5.4,±4.3,±4.1,±4.8,...,±6.5,±5.5,±1.8,±1.3,NaN,±7.7,±6.7,±5.9,±2.3,±81
996,"Pasco County, Florida!!Owner-occupied housing ...","72,592",NaN,9.5%,22.9%,19.8%,18.6%,17.2%,9.5%,2.5%,...,30.0%,10.2%,2.9%,1.1%,NaN,23.7%,25.0%,48.8%,2.4%,"1,503"
997,"Pasco County, Florida!!Owner-occupied housing ...","±2,756",NaN,±1.4,±2.0,±2.3,±2.3,±2.2,±1.6,±0.6,...,±2.2,±1.6,±0.7,±0.6,NaN,±2.1,±2.2,±2.4,±0.7,±89


In [261]:
housing.columns

Index(['COUNTY', 'N', 'VALUE', 'Less than $50,000', '$50,000 to $99,999',
       '$100,000 to $149,999', '$150,000 to $199,999', '$200,000 to $299,999',
       '$300,000 to $499,999', '$500,000 or more', 'MEDIAN_VAL',
       'MORTGAGE STATUS',
       '    With either a second mortgage, or home equity loan, but not both',
       '        Second mortgage only', '        Home equity loan only',
       '    Both second mortgage and home equity loan',
       '    No second mortgage and no home equity loan',
       'HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2013 INFLATION-ADJUSTED DOLLARS)',
       'Less than $10,000', '$10,000 to $24,999', '$25,000 to $34,999',
       '$35,000 to $49,999', '$50,000 to $74,999', '$75,000 to $99,999',
       '$100,000 to $149,999', '$150,000 or more',
       '    Median household income (dollars)',
       'RATIO OF VALUE TO HOUSEHOLD INCOME IN THE PAST 12 MONTHS',
       'Less than 2.0', '2.0 to 2.9', '3.0 to 3.9', '4.0 or more',
       '    Not computed', '

In [267]:
# formatting county names
housing.COUNTY = housing.COUNTY.replace({'!!Owner-occupied housing units with a mortgage!!':' ',
                                        'Estimate':'EST','Margin of Error':'MAR',
                                        'County,':'',
                                        ',':''}, regex=True,)

state_abb = {'Alabama':'AL','Alaska': 'AK','Arizona':'AZ','Arkansas':'AR',
             'California':'CA','Colorado':'CO','Conneticut':'CT','Delaware':'DE',
             'Florida':'FL','Georgia':'GA','Hawaii':'HI','Idaho':'ID','Illinois':'IL',
             'Indiana':'IN','Iowa':'IA','Kansas':'KS','Kentucky':'KY','Louisiana':'LA',
             'Maine':'ME','Maryland':'MD','Massachusetts':'MA','Michigan':'MI','Minnesota':'MN',
             'Mississippi':'MS','Missouri':'MO','Montana':'MT','Nebraska':'NE',
             'Nevada':'NV','New Hampshire':'NH','New Jersey':'NJ','New Mexico':'NM',
             'New York':'NY','North Carolina':'NC','North Dakota':'ND','Ohio':'OH',
             'Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI',
             'South Carolina':'SC','South Dakota':'SD','Tennessee':'TN','Texas':'TX',
             'Utah':'UT','Vermont':'VT','Virginia':'VA','Washington':'WA',
             'West Virginia':'WV','Wisconsin':'WI','Wyoming':'WY'}

housing.COUNTY = housing.COUNTY.replace(state_abb,regex=True)

In [268]:
# creating column for STATE
housing['STATE'] = housing.COUNTY.apply(lambda x: str(x)[-6:-3])

housing['STATE'] = housing['STATE'].str.upper()
housing.head()

Label,COUNTY,N,VALUE,"Less than $50,000","$50,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 to $299,999","$300,000 to $499,999","$500,000 or more",...,20 to 29 percent,30 percent or more,Zero or negative income,REAL ESTATE TAXES,Less than $800,"$800 to $1,499","$1,500 or more",No real estate taxes paid,Median (dollars),STATE
0,Fairbanks North Star Borough AK EST,"13,579",NaN,1.3%,0.8%,10.1%,28.9%,41.1%,12.1%,5.6%,...,N,N,N,NaN,3.3%,8.9%,78.4%,9.4%,"2,969",AK
1,Fairbanks North Star Borough AK MAR,"±1,488",NaN,±1.2,±1.2,±4.7,±6.7,±6.4,±5.2,±4.0,...,N,N,N,NaN,±2.3,±4.1,±6.3,±4.0,±272,AK
2,Apache AZ EST,"4,119",NaN,16.5%,21.4%,12.9%,27.0%,11.1%,5.1%,5.9%,...,4.5%,0.0%,0.8%,NaN,37.3%,16.9%,8.5%,37.2%,690,AZ
3,Apache AZ MAR,±842,NaN,±6.5,±7.5,±7.7,±12.5,±6.9,±4.6,±5.1,...,±5.4,±4.0,±0.9,NaN,±13.6,±10.1,±7.1,±10.1,±255,AZ
4,Yuba CA EST,"9,869",NaN,4.9%,13.3%,18.5%,26.3%,23.2%,10.9%,3.0%,...,N,N,N,NaN,11.7%,26.2%,61.2%,0.8%,"1,856",CA


In [256]:
# double checking state column, hmm only 48 states?
housing['STATE'].unique()
len(housing['STATE'].unique())

48

In [270]:
housing['MEDIAN_VAL']

0      224,000
1      ±20,267
2      145,900
3      ±42,512
4      173,600
        ...   
994    115,200
995    ±19,775
996    143,600
997     ±7,379
998    118,300
Name: MEDIAN_VAL, Length: 999, dtype: object

In [271]:
# before I continue, let's get rid of some of the columns that i dont need
house = housing[['COUNTY','N','STATE','MEDIAN_VAL']] 
house.head()

Label,COUNTY,N,STATE,MEDIAN_VAL
0,Fairbanks North Star Borough AK EST,"13,579",AK,"224,000"
1,Fairbanks North Star Borough AK MAR,"±1,488",AK,"±20,267"
2,Apache AZ EST,"4,119",AZ,"145,900"
3,Apache AZ MAR,±842,AZ,"±42,512"
4,Yuba CA EST,"9,869",CA,"173,600"


In [221]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

states = ["AL", "AK", "AZ", "AR", "", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "", "UT", "VT", "", "WA", "WV", "WI", "WY"]

In [276]:
# hmm, seem to be only 47 states made it into the df... is that accurate???
len(house['STATE'].unique())

47

In [277]:
# first let's identify which states aren't included.
house['STATE'].value_counts()

TX     72
VA     60
CA     52
NY     52
GA     50
PA     46
OH     40
NC     40
FL     38
IL     32
WI     28
SC     28
WA     28
IN     26
AL     26
MI     26
MD     26
TN     24
CO     22
NJ     22
KY     20
LA     18
UT     18
MO     17
OR     16
NM     14
OK     14
IA     14
AR     12
MS     12
MN     12
MA     12
KS     10
NH     10
AZ     10
HI      8
MT      8
ND      6
ID      6
RI      4
SD      4
ME      4
NE      4
AK      2
DE      2
NV      2
WY      2
Name: STATE, dtype: int64

It's been difficult to find housing data for 1993, so I think I might just update the urban-rural continuum codes so that I can use housing prices that are more recent. I have 2013 codes [here](https://www.ers.usda.gov/data-products/rural-urban-continuum-codes.aspx)

In [283]:
# importing RUC_2013
ruc_13 = pd.read_csv('/Users/patricknorman/Documents/GitHub/metis-project-2/data/ruc_13/ruc_2013.csv')
ruc_13.head()

,FIPS,State,County_Name,Population_2010,RUCC_2013,Description
0,1001,AL,Autauga County,"54,571",2.0,"Metro - Counties in metro areas of 250,000 to ..."
1,1003,AL,Baldwin County,"182,265",3.0,Metro - Counties in metro areas of fewer than ...
2,1005,AL,Barbour County,"27,457",6.0,"Nonmetro - Urban population of 2,500 to 19,999..."
3,1007,AL,Bibb County,"22,915",1.0,Metro - Counties in metro areas of 1 million p...
4,1009,AL,Blount County,"57,322",1.0,Metro - Counties in metro areas of 1 million p...


In [284]:
nat.head()

,FIPS,COMBINED_COUNTY_INDEX,STATE,COUNTY,CENSUS_DIV,RUC_93,UIC_93,JAN_TEMP,JAN_SUN,JULY_TEMP,...,WATER,LN_WATER,JAN TEMP - Z,JAN SUN - Z,JUL TEMP - Z,JUL HUM - Z,TOPOG - Z,LN WATER AREA - Z,NAT_AMENITY,RANK
0,1001,1001,AL,AUTAUGA,6,2,2,47.4,130,81.0,...,1.40,4.944,1.20277,-0.64758,-0.13206,-0.68502,0.77701,0.26585,0.78,4
1,1003,1003,AL,BALDWIN,6,2,2,51.9,152,80.6,...,21.24,7.661,1.57504,0.01482,0.36308,-1.09576,-0.73966,1.70428,1.82,4
2,1005,1005,AL,BARBOUR,6,6,5,48.8,152,81.0,...,2.17,5.378,1.31859,0.01482,-0.01148,-0.89039,-0.73966,0.49570,0.19,4
3,1007,1007,AL,BIBB,6,6,6,44.8,130,80.8,...,0.50,3.916,0.98768,-0.64758,-0.30223,-0.68502,0.77701,-0.27812,-0.15,3
4,1009,1009,AL,BLOUNT,6,2,2,41.9,130,78.4,...,0.76,4.336,0.74778,-0.64758,0.09330,-0.68502,0.77701,-0.05592,0.23,4


In [286]:
nat_updated = pd.merge(nat,ruc_13['RUCC_2013'],on=nat['FIPS'])

KeyError: 0        1001
1        1003
2        1005
3        1007
4        1009
        ...  
3106    56037
3107    56039
3108    56041
3109    56043
3110    56045
Name: FIPS, Length: 3111, dtype: int64